In [3]:
# !pip install spotipy

In [40]:
import operator
import sys
import pyspark
from pyspark.sql import SparkSession, types
from pyspark import SparkConf, SparkContext
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import pandas as pd

In [20]:
CLIENT_ID = 'd3e048fac0e8438cab168ac60459044c'
CLIENT_SECRET = '4d47925e9199481fb41cc0c1406de938'

In [21]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
cid = CLIENT_ID
secret = CLIENT_SECRET
client_credentials_manager = SpotifyClientCredentials(
    client_id=cid, client_secret=secret)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

In [25]:
# create lists of columns to be used when reading/merging the csv's
columns = ['title', 'rank', 'date', 'artist', 'url', 'region', 'chart', 'trend', 'streams']
DATA_PATH = "gs://big_data_spotify_bucket/charts.csv"
# Spark settings
sc = SparkContext.getOrCreate()
spark = SparkSession(sc)
# sp.audio_features(track_url)[0]

In [32]:
# Load the data, cache this since we're accessing this each iteration
data = spark.read.format("csv").option("header", "true").load(DATA_PATH).cache()
#===============================

22/12/15 02:47:29 WARN org.apache.spark.sql.execution.CacheManager: Asked to cache already cached data.


In [58]:
data = data.withColumn("rank", data["rank"].cast("int")).withColumn("streams", data["streams"].cast("int"))
data.printSchema()
data.show(20)

root
 |-- title: string (nullable = true)
 |-- rank: integer (nullable = true)
 |-- date: string (nullable = true)
 |-- artist: string (nullable = true)
 |-- url: string (nullable = true)
 |-- region: string (nullable = true)
 |-- chart: string (nullable = true)
 |-- trend: string (nullable = true)
 |-- streams: integer (nullable = true)

+--------------------+----+----------+--------------------+--------------------+---------+------+-------------+-------+
|               title|rank|      date|              artist|                 url|   region| chart|        trend|streams|
+--------------------+----+----------+--------------------+--------------------+---------+------+-------------+-------+
|Chantaje (feat. M...|   1|2017-01-01|             Shakira|https://open.spot...|Argentina|top200|SAME_POSITION| 253019|
|Vente Pa' Ca (fea...|   2|2017-01-01|        Ricky Martin|https://open.spot...|Argentina|top200|      MOVE_UP| 223988|
|Reggaetón Lento (...|   3|2017-01-01|                CNCO|

In [57]:

# data.take(1)
url_column = data.select('url')
print(url_column.take(1)[0][0])
sp.audio_features(url_column.take(1)[0][0])[0]

https://open.spotify.com/track/6mICuAdrwEjh6Y6lroV2Kg


{'danceability': 0.852,
 'energy': 0.773,
 'key': 8,
 'loudness': -2.921,
 'mode': 0,
 'speechiness': 0.0776,
 'acousticness': 0.187,
 'instrumentalness': 3.05e-05,
 'liveness': 0.159,
 'valence': 0.907,
 'tempo': 102.034,
 'type': 'audio_features',
 'id': '6mICuAdrwEjh6Y6lroV2Kg',
 'uri': 'spotify:track:6mICuAdrwEjh6Y6lroV2Kg',
 'track_href': 'https://api.spotify.com/v1/tracks/6mICuAdrwEjh6Y6lroV2Kg',
 'analysis_url': 'https://api.spotify.com/v1/audio-analysis/6mICuAdrwEjh6Y6lroV2Kg',
 'duration_ms': 195840,
 'time_signature': 4}

SyntaxError: invalid syntax (1262584681.py, line 1)

In [5]:
artist_popularity = []
artist_genres = []
artist_followers = []
for a_id in track_df.artist_id:
  artist = sp.artist(a_id)
  artist_popularity.append(artist['popularity'])
  artist_genres.append(artist['genres'])
  artist_followers.append(artist['followers']['total'])

1000